## **텍스트마이닝(윤상혁 교수)**
## ch3. 토픽모델링
## ch3_01. LDA토픽모델링


---

In [ ]:
# konlpy 패키지 설치: 한국어 형태소 분석을 위한 라이브러리입니다.
!pip install konlpy

In [ ]:
# pandas 라이브러리를 pd라는 이름으로 임포트: 데이터 처리를 위한 주요 라이브러리입니다.
import pandas as pd

# konlpy 라이브러리에서 Okt 형태소 분석기를 임포트: 한국어 텍스트를 형태소 단위로 분석합니다.
from konlpy.tag import Okt

# 엑셀 파일을 데이터프레임 형태로 불러오기: 분석할 뉴스 데이터를 불러옵니다.
df = pd.read_excel('/content/빅카인즈_주식_뉴스.xlsx')


In [ ]:
df

In [ ]:
# '제목'과 '본문' 컬럼을 합쳐 '제목_본문'이라는 새로운 컬럼을 생성합니다.
df['제목_본문'] = df['제목'] + df['본문']

In [ ]:
df['제목_본문']

In [ ]:
# Okt 형태소 분석기 객체를 생성합니다. 한국어 텍스트 분석을 위한 준비 단계입니다.
okt = Okt()

# 텍스트 데이터를 형태소 단위로 분석하여 특정 품사(명사, 형용사, 동사)만 추출하는 함수입니다.
def tokenizer(text):
    morph = okt.pos(text)
    words = []
    for word, tag in morph:
        if tag in ['Noun', 'Adjective', 'Verb']:
            if len(word) > 1:  # 한 글자보다 긴 단어만 선택
                words.append(word)
    return words

In [ ]:
text='나는 어제 영어 공부를 열심해 했다'

In [ ]:
okt = Okt()
okt.pos(text)

In [ ]:
# 데이터프레임의 모든 데이터를 문자열 타입으로 변환합니다. 이는 형태소 분석을 위한 준비 단계입니다.
df = df.astype('str')

#'제목_본문' 컬럼에 tokenizer 함수를 적용하여, 형태소 분석을 수행하고 결과를 다시 해당 컬럼에 저장합니다.
df['제목_본문'] = df['제목_본문'].apply(tokenizer)

# 데이터프레임의 모든 열을 문자열 타입으로 변환합니다. 안전한 파일 저장을 위한 단계입니다.
df = df.astype('str')

# 데이터프레임을 '정제파일.csv'로 저장합니다. 분석 결과를 파일로 저장하는 단계입니다.
df.to_csv('정제파일.csv', encoding='utf-8')

In [ ]:
df['제목_본문']

# 빈도분석 및 워드크라우드 시각화

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  # CountVectorizer 임포트
df = pd.read_csv('/content/정제파일.csv', encoding='utf-8')
#불용어를 처리합니다.
stop_words=["코스피", "증시"]

cv = CountVectorizer(ngram_range=(1, 1), max_features=3000, stop_words=stop_words)###최대개수설정

# CountVectorizer를 사용하여 '제목_본문' 컬럼의 텍스트 데이터를 문서-단어 행렬로 변환합니다.
tdm = cv.fit_transform(df['제목_본문'])

# 단어와 그 빈도수를 담은 데이터프레임을 생성합니다. 이는 단어 사용 빈도 분석을 위한 단계입니다.
word_count_tf = pd.DataFrame({'단어': cv.get_feature_names_out(), '빈도': tdm.sum(axis=0).flat})

# 단어 빈도수를 기준으로 내림차순으로 정렬합니다. 이는 가장 자주 사용된 단어를 확인하기 위한 단계입니다.
word_count_tf = word_count_tf.sort_values('빈도', ascending=False)

# 단어 빈도수를 'word_count.xlsx' 엑셀 파일로 저장합니다. 결과를 저장하는 단계입니다.
word_count_tf.to_excel('word_count.xlsx')

In [ ]:
# WordCloud 라이브러리를 임포트합니다: 텍스트 데이터의 빈도수를 시각화하기 위한 도구입니다.
from wordcloud import WordCloud

# 한글 폰트 경로를 지정합니다. WordCloud에서 한글을 정상적으로 표시하기 위한 준비 단계입니다.
font_path = '/content/malgun.ttf'

# WordCloud 객체를 생성합니다. 여기서는 배경색을 흰색으로, 최대 단어 수를 50개로 설정합니다.
wc = WordCloud(background_color='white', max_words=50, font_path=font_path,width=400, height=400)

# 단어 빈도 사전을 생성합니다. 이는 WordCloud 생성에 사용될 데이터입니다.
count_dic = dict(zip(word_count_tf['단어'], word_count_tf['빈도']))

# WordCloud를 이용해 단어 빈도를 시각화합니다. 이는 가장 많이 사용된 단어를 시각적으로 확인하기 위한 단계입니다.
cloud = wc.fit_words(count_dic)
cloud.to_file('word_cloud.png')
cloud.to_image()

# TF-IDF 및 워드크라우드 시각화

In [ ]:
# TfidfTransformer를 임포트합니다: TF-IDF 값을 계산하기 위한 도구입니다.
from sklearn.feature_extraction.text import TfidfTransformer

# TfidfTransformer 객체를 생성합니다. 이는 문서-단어 행렬의 단어 빈도수를 TF-IDF 값으로 변환하기 위한 준비 단계입니다.
trans = TfidfTransformer()

# 문서-단어 행렬에 대해 TF-IDF 변환을 수행합니다. 이는 각 단어의 중요도를 수치화하는 단계입니다.
dtm2 = trans.fit_transform(tdm)

# 단어와 그 TF-IDF 값을 담은 데이터프레임을 생성합니다. 이는 단어의 중요도 분석을 위한 단계입니다.
df2 = pd.DataFrame({'단어': cv.get_feature_names_out(), 'tf-idf': dtm2.sum(axis=0).flat})

# TF-IDF 값을 기준으로 내림차순으로 정렬합니다. 이는 중요한 단어를 확인하기 위한 단계입니다.
df2 = df2.sort_values('tf-idf', ascending=False)

# 인덱스를 재설정합니다. 이는 데이터의 가독성을 높이기 위한 단계입니다.
df2 = df2.reset_index(drop=True)
df2.index = df2.index + 1

# TF-IDF 값을 'tf_idf.csv' 파일로 저장합니다. 결과를 저장하는 단계입니다. 인코딩을 cp949로 설정하여 한글이 깨지지 않게 합니다.
df2.to_csv('tf_idf.csv', encoding='cp949')

In [ ]:
from wordcloud import WordCloud  # WordCloud 임포트

# 한글 폰트 경로 지정
font_path = '/content/malgun.ttf'

# WordCloud 객체를 다시 생성합니다. 이번에는 TF-IDF 값을 기반으로 가장 중요한 단어를 시각화합니다.
wc = WordCloud(background_color='black', max_words=30, font_path=font_path,width=400, height=400)

# TF-IDF 값을 기반으로 단어 빈도 사전을 생성합니다.
count_dic = dict(zip(df2['단어'], df2['tf-idf']))

# WordCloud를 이용해 TF-IDF 값을 시각화합니다. 이는 중요한 단어를 시각적으로 확인하기 위한 단계입니다.
cloud = wc.fit_words(count_dic)
cloud.to_image()

# LDA분석

In [ ]:
!pip install pyldavis

In [ ]:
# LDA 분석을 위한 라이브러리 임포트
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pandas as pd

In [ ]:
# CSV 파일에서 데이터 불러오기
data = pd.read_csv('/content/정제파일.csv', encoding='utf-8')
data = data.astype('str')
data=data['제목_본문']

In [ ]:
data

In [ ]:
# 'data' 리스트에 저장된 텍스트 데이터(예: '제목_본문')를 공백 기준으로 분할하여 토큰 리스트를 생성합니다.
token = [keywords.split() for keywords in data]
# 토큰 리스트를 기반으로 Gensim의 Dictionary 객체를 생성합니다. 이 객체는 각 토큰에 고유한 ID를 매핑합니다.
# 결과적으로, 텍스트 데이터의 모든 유니크한 단어가 ID와 함께 딕셔너리로 관리됩니다.
id2word = corpora.Dictionary(token)

# 딕셔너리에서 빈도수 기반으로 단어를 필터링합니다. no_below와 no_above 파라미터를 사용하여,
# 너무 드물게 나타나는 단어(no_below 이하)와 너무 자주 나타나는 단어(no_above 이상)를 제거합니다.
# no_below=3는 각 단어가 최소 3개의 문서에 나타나야 함을 의미합니다.
# no_above=0.1는 전체 문서의 10% 미만에만 나타나는 단어를 유지함을 의미합니다.
id2word.filter_extremes(no_below=5, no_above=0.2)

# 최종적으로, 각 문서를 (단어 ID, 단어 빈도) 튜플의 리스트로 변환하는 문서-단어 빈도 매트릭스(corpus)를 생성합니다.
# 이 과정에서 각 문서 내에 있는 단어들이 얼마나 자주 나타나는지를 기록합니다.
# 이 corpus는 텍스트 데이터를 기반으로 한 다양한 모델링 작업(예: LDA 주제 모델링)에 사용될 수 있습니다.
corpus = [id2word.doc2bow(text) for text in token]
corpus

In [ ]:
# numpy 라이브러리를 np라는 이름으로 가져옵니다. numpy는 다양한 수학적 연산을 위해 사용됩니다.
import numpy as np

perplexity_values = []

# 토픽의 수를 3에서 9까지 변화시키면서 LDA 모델의 퍼플렉시티를 계산합니다.
# 토픽 수가 변할 때마다 모델의 성능을 퍼플렉시티를 통해 평가합니다.
for i in range(3, 10):
    # LDA 모델을 생성합니다. 'num_topics=i'는 토픽의 수를 i로 설정합니다.
    # 'id2word'는 단어의 인덱스 매핑을 나타내는 사전입니다.
    # 'random_state=100'는 모델 결과의 재현 가능성을 위해 난수 생성기의 시드를 설정합니다.
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=id2word, random_state=100)

    # 모델의 로그 퍼플렉시티를 계산하고, 이를 자연로그의 밑인 e를 기반으로 하는 지수 함수로 변환하여
    # 퍼플렉시티를 계산합니다. 퍼플렉시티 값이 낮을수록 모델이 데이터를 더 잘 설명하고 있다는 의미입니다.
    perplexity = np.exp(ldamodel.log_perplexity(corpus))

    # 계산된 퍼플렉시티를 리스트에 추가합니다.
    perplexity_values.append(perplexity)

In [ ]:
# Perplexity 그래프 시각화
import matplotlib.pyplot as plt

x = range(3, 10)
plt.plot(x, perplexity_values)
plt.xlabel("number of topics")
plt.ylabel("perplexity score")
plt.show()

In [ ]:
from gensim.models import CoherenceModel  # CoherenceModel 임포트

coherence_values = []
# 다양한 토픽 수에 대해 Coherence 계산
for i in range(3, 10):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=id2word,random_state=100)
    coherence_model_lda = CoherenceModel(model=ldamodel, texts=token, dictionary=id2word, topn=10)
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

In [ ]:
x = range(3, 10)
plt.plot(x, coherence_values)
plt.xlabel("number of topics")
plt.ylabel("coherence score")
plt.show()

In [ ]:
k=4 ###적정토픽수 입력 coherence를 최대로 하고 perpelxity를 최소로 하는 적절합 토픽수

In [ ]:
#적정토픽에 해당 되는 단어를 구하기!
from gensim import corpora, models
# 적절한 토픽 수를 설정하여 LDA 모델 훈련
model = LdaModel(corpus=corpus, num_topics=k, id2word=id2word, passes=15,random_state=100)  # num_topic에 적절 토픽 수 설정
topics = model.print_topics(num_words=15) ####토픽당 단어수설정

In [ ]:
topics

In [ ]:
# 도출된 토픽을 CSV 파일로 저장
TOPIC={'topics':topics}
TOPIC=pd.DataFrame(TOPIC)
TOPIC.to_csv('lda_result.csv',encoding='cp949')

In [ ]:
!pip install pyldavis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook() # Jupyter Notebook에서 pyLDAvis 시각화를 사용할 수 있도록 설정
# pyLDAvis를 이용해 LDA 모델 시각화
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis  # 시각화 객체 출력